In [ ]:
# Starter imports
import hashlib, secrets, hmac

# TODO: add demos for hashing, HMAC, and simple token validation

## Hashing and HMAC
- Hashes ensure integrity; HMACs ensure integrity plus authenticity with a secret key.
- Always compare digests with constant-time checks.

In [ ]:
import hashlib, hmac, secrets

message = b"important message"
key = secrets.token_bytes(32)

digest = hashlib.sha256(message).hexdigest()
mac = hmac.new(key, message, hashlib.sha256).hexdigest()

hmac.compare_digest(mac, hmac.new(key, message, hashlib.sha256).hexdigest())


In [ ]:
password = b"s3cret"
salt = secrets.token_bytes(16)
dk = hashlib.pbkdf2_hmac("sha256", password, salt, iterations := 100_000, dklen=32)
len(dk), iterations

## Auth vs AuthZ, Inputs, and Secrets
- Authentication proves identity; authorization checks permissions.
- Validate/encode inputs (e.g., parameterized SQL, escaping) to avoid injection.
- Use strong randomness for tokens; never roll your own crypto.

In [ ]:
def hash_password(password: bytes, salt: bytes | None = None, iterations: int = 120_000):
    salt = salt or secrets.token_bytes(16)
    dk = hashlib.pbkdf2_hmac("sha256", password, salt, iterations, dklen=32)
    return {"salt": salt, "iterations": iterations, "dk": dk}

def verify_password(password: bytes, record: dict):
    candidate = hashlib.pbkdf2_hmac("sha256", password, record["salt"], record["iterations"], dklen=32)
    return hmac.compare_digest(candidate, record["dk"])

record = hash_password(b"p@ssw0rd")
verify_password(b"p@ssw0rd", record), verify_password(b"wrong", record)

## Broader Security Notes
- CIA triad: confidentiality, integrity, availability; design controls per axis.
- Common pitfalls: injection, broken auth/session, XSS/CSRF, insecure deserialization, sensitive data exposure.
- Secrets management: prefer env/secret stores, short-lived tokens, key rotation; avoid hard-coding secrets.

In [ ]:
# Simple signed token with HMAC
import base64, json, hmac, hashlib, secrets

SECRET = secrets.token_bytes(32)

def make_token(payload: dict) -> str:
    body = json.dumps(payload, separators=(",", ":"), sort_keys=True).encode()
    sig = hmac.new(SECRET, body, hashlib.sha256).digest()
    return base64.urlsafe_b64encode(body + b"." + sig).decode()

def verify_token(token: str) -> dict | None:
    raw = base64.urlsafe_b64decode(token.encode())
    body, sig = raw.rsplit(b".", 1)
    expected = hmac.new(SECRET, body, hashlib.sha256).digest()
    if not hmac.compare_digest(expected, sig):
        return None
    return json.loads(body)

sample = {"sub": "user123", "exp": 9999999999}
tok = make_token(sample)
verify_token(tok)